## 7-2 機器の異常検知

### ライブラリーのインポート

In [ ]:
import numpy as np
import pandas as pd
import itertools
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

- itertools
  - 効率的なループ実行のためのイテレータを生成するライブラリ
  - イテレータとは複数の値を持つデータを順番にアクセスするための機能
- sklearn.svm.OneClassSVM
  - One-class SVMの機械学習ライブラリ
- sklearn.preprocessing.StandardScaler
  - データの標準化（平均が0で分散が1のデータに変換する操作）のための関数
- sklearn.metrics
  - precision_recall_fscore_support：適合率、再現率、Fスコアを求める関数
  - confusion_matrix：混同行列を求める関数

### 訓練データの読み込み

In [ ]:
x_train = pd.read_csv('data/anomaly_train.csv')
print('(count, features) = (%d, %d)' % x_train.shape)
x_train.sample(3)

- 訓練データを読み込んで、件数・項目数およびサンプル3件を出力する。

### 検証用データとテストデータの読み込み

In [ ]:
# 検証用データ
x_valid = pd.read_csv('data/anomaly_valid.csv')
label_valid = pd.read_csv('data/anomaly_valid_label.csv')
print('Validation data (normal, anomaly, total) = (%d, %d, %d)' %
      ((label_valid == 1).sum(), (label_valid == -1).sum(), 
       x_valid.shape[0]))

# テストデータ
x_test = pd.read_csv('data/anomaly_test.csv')
label_test = pd.read_csv('data/anomaly_test_label.csv')
print('Test data (normal, anomaly, total) = (%d, %d, %d)' %
      ((label_test == 1).sum(), (label_test == -1).sum(), 
       x_test.shape[0]))

- 検証用データとテストデータを読み込んで、それぞれの正常・異常データ数と合計を出力する。
  - 検証用データ：機械学習のハイパーパラメータの最適値を検証するためのデータ
  - テストデータ：学習済みモデルの評価するためのデータ

### データの標準化

In [ ]:
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.fit_transform(x_test)
x_valid_std = sc.fit_transform(x_valid)

- sc = StandardScaler()
  - データ標準化処理のインスタンスの生成
- sc.fit_transform(･･･)
  - データ標準化の実行
  - 標準化：（データ - 平均値）/ 標準偏差

### ハイパーパラメータの探索

In [ ]:
# パラメータ（nu、gamma）の範囲の設定
nu =[0.001, 0.01, 0.1]
gamma = [0.1, 0.5, 1.0]

# Fスコアが最大となるパラメータの探索
cnt, idx, f_score = 0, [], []
for n, r in itertools.product(nu, gamma):
    cnt += 1
    print('Parameter Search', cnt)
    ocsvm = OneClassSVM(kernel='rbf', nu=n, gamma=r)
    ocsvm.fit(x_train_std)
    prec_rec_f = precision_recall_fscore_support(
        label_valid, ocsvm.predict(x_valid_std))
    f_score.append(np.average(prec_rec_f[2]))
    idx.append([n, r])

best_idx = idx[np.argmax(f_score)]
print('Best Parameter (nu=%.4f, gamma=%.4f)' % (best_idx[0], best_idx[1]))

- OneClassSVMのパラメータ（nu、gamma）を変化させ検証用データに対する最適値を探索する。
  - nu：異常データの割合を決めるパラメータ
  - gamma：分類の複雑さのパラメータ（カーネル係数）
  - idxリスト：nuとgammmaの組み合わせ記録用リスト
  - f_scoreリスト：Fスコアの記録用リスト
- for n, r in itertools.product(nu, gamma):
  - 各パラメータの要素を一つずつ取り出した組み合わせでループ処理する。
- ocsvm = OneClassSVM(kernel='rbf', nu=n, gamma=r)
  - OneClassSVM学習のインスタンスを生成する。
- ocsvm.fit(x_train_std)
  - OneClassSVMによる学習を実行する。

### 機械学習実行と評価

In [ ]:
# 機械学習の実行
ocsvm = OneClassSVM(kernel='rbf', nu=best_idx[0], gamma=best_idx[1])
ocsvm.fit(x_train_std)

# テストデータによる評価
y_pred = ocsvm.predict(x_test_std)
prec_rec_f = precision_recall_fscore_support(label_test, y_pred)
print('Precision %.4f, Recall %.4f, F-score %.4f' % 
      (np.average(prec_rec_f[0]),    # Precision
       np.average(prec_rec_f[1]),    # Recall
       np.average(prec_rec_f[2])))   # F-score

# 混同行列の表示
print('Confusion Matrix')
df = pd.DataFrame(confusion_matrix(label_test, y_pred))
df.columns = ['anomaly', 'normal']
print(df)

- 探索により求められた最適なハイパーパラメータ（nu, gamma）で機械学習を実行する。
- 学習実行後に、テストデータにより評価する。
  - precision_recall_fscore_support：適合率、再現率、Fスコアを求める。
  - confusion_matrix：混同行列を求める。